In [1]:
import pandas as pd,numpy as np,matplotlib.pyplot as plt, time  
from numpy import *  
from random import seed
from random import randrange
from csv import reader
from math import sqrt
from sklearn import preprocessing
%matplotlib inline

def sigmoid(x):  
    return 1.0 / (1 + np.exp(-x))  

def prepareData(train_pd, stance):  
    train_x = train_pd[selected_features].values
    train_y = train_pd['Stance'].values == stance
    return mat(train_x), mat(train_y).transpose()

def predict(weights, test_x):
    predict = sigmoid(test_x * weights)
    return predict

def gradient_descent(train_x, train_y, weights, alpha, iteration):  
    for k in range(iteration):  
        output = sigmoid(np.dot(train_x, weights))  
        loss = train_y - output
        weights = weights + alpha * train_x.transpose() * loss 
    
    mse = (sum(loss))**2/len(train_x)
    print('mse:',mse)

    return weights

In [2]:
selected_features = ['cos','dis','overlap_count'] # 0.3621


load_feature_df = pd.read_csv('train_processed_feat.csv')

train_x_unrelated, train_y_unrelated = prepareData(load_feature_df, 'unrelated')
train_x_agree, train_y_agree = prepareData(load_feature_df, 'agree')
train_x_disagree, train_y_disagree = prepareData(load_feature_df, 'disagree')
train_x_discuss, train_y_discuss = prepareData(load_feature_df, 'discuss')

# train logistic regression
W = np.zeros((len(selected_features),1))
alpha = 0.02
print('start train using logistic regression:')
W_unrelated = gradient_descent(train_x_unrelated, train_y_unrelated, W, alpha, 3000)
W_agree = gradient_descent(train_x_agree, train_y_agree, W, alpha, 3000)
W_disagree = gradient_descent(train_x_disagree, train_y_disagree, W, alpha, 3000)
W_discuss = gradient_descent(train_x_discuss, train_y_discuss, W, alpha, 3000)

# predict on validation 
validation_data = pd.read_csv('test_processed_feat.csv')
validation_x = validation_data[selected_features].values

validation_predict_agree = predict(W_agree, validation_x)*0.072
validation_predict_disagree = predict(W_disagree, validation_x)*0.017
validation_predict_discuss = predict(W_discuss, validation_x)*0.176
validation_predict_unrelated = predict(W_unrelated, validation_x)*0.735

# mayWe threshold the unrelated? and select the West among the other if Welow the threshold?
prediction = np.column_stack([validation_predict_agree,\
                              validation_predict_disagree,\
                              validation_predict_discuss,\
                              validation_predict_unrelated])
label_dict = {0:'agree', 1:'disagree', 2:'discuss',3:'unrelated'}
prediction = np.array([label_dict[result.argmax()] for result in prediction])


# evaluate accuracy and score
unrelated_correct = 0
other_correct = 0
N_unrelated = len(validation_data[validation_data.Stance == 'unrelated'])
N_other = len(validation_data) - N_unrelated

for p,true_laWel in zip(prediction, validation_data.Stance):
    if p == true_laWel:
        if p == 'unrelated':
            unrelated_correct +=1
        else:
            other_correct +=1

print('evaluate score:',unrelated_correct/N_unrelated * 0.25 + 0.75* other_correct/N_other)
print('find unrelated:',unrelated_correct,'/' ,N_unrelated, 'accuracy:', unrelated_correct/N_unrelated)
print('find other:',other_correct,'/',N_other, 'accuracy:', other_correct/N_other)

start train using logistic regression:


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in exp
  # This is added back by InteractiveShellApp.init_path()


mse: 2088.57885121
mse: 2804.57586851
mse: 1831.04387237
mse: 394.877464685
evaluate score: 0.6831160609565375
find unrelated: 17222 / 18349 accuracy: 0.9385797591149382
find other: 4224 / 7064 accuracy: 0.5979614949037373


# select features

In [3]:
w = pd.DataFrame(np.column_stack([W_unrelated, W_agree, W_disagree,W_discuss]), columns=['unrelated', 'agree', 'disagree', 'discuss'])
w

,unrelated,agree,disagree,discuss
0,-5572.542506,2449.852058,253.623290,8132.257128
1,197.702580,-59.897257,-41.346767,-1149.793263
2,50.435724,-207.642582,-359.361429,162.223508


In [4]:
a = w.values
a = [abs(v) for v in a]
np.sum(a, axis = 1)

array([ 16408.27498212,   1448.73986788,    779.66324373])

In [5]:
np.sum(a, axis = 1).argmin()

2

In [6]:
import os
os.system('say "you are very beautiful"')

0